# Markov Decision Process

Die Grundlage für Reinforcement learning bilden Markow-Entscheidungsprozesse. Aus diesem Grund sollen jene nun genauer betrachtet werden.
Ein Markov Decision Process (MDP) oder zu Deutsch auch Markow-Entscheidungsprozess ist ein Fünf-Tupel der Form $(S, A, T, R, s_0) $
wobei gilt:

* $S$ = eine Menge von Zuständen (states)
* $A$ = eine Menge von Aktionen (actions)
* $T(s, a, s’)$ = eine transition function mit $s \in S$ 
* $R$ = Belohnungsfunktion $R(s, a, s')$
* $s_0$ = Startzustand

Ein MDP ist immer von seinem Umfeld, dem Environment abhänging und kann optional einen Endzustand besitzen. Die Lösung von einem solchen Problem ist eine Funktion der Form $\pi \colon S\rightarrow A$. Diese wird auch Policy genannt. Die Policy gibt an, welche Aktion in welchem Zustand ausgeführt wird. Ziel ist es, eine (möglichst) optimale Policy zu finden. (vgl. Abbeel und Klein)

$States$ sind eine Menge an Token, die angeben, in welchem Zustand sich jemand in der Umgebung befindet. Dies kann zum Beispiel die Position sein, in der sich jemand in der Umgebung befindet. (im Beispiel 12 verschiedene Zustände) Ein $Agent$ kann in einer $Umgebung$ (Enviroment e) fest definierte Menge an $Aktionen$ ausführen, die ihn in einen neuen State bringen.

Die $Transistion-Funktion$, auch Model genannt, der Form $T(s,a,s’)$ bekommt einen State s, eine Aktion a und einen weiteren State s’ und berechnet die Wahrscheinlichkeit, dass die Aktion a im State s den Agenten in den State s’ überführt. (wie oft welche aktionen ausgeführt werden ist definiert) (terministisch, unterministisch) Das Model genügt zwei Eigenschaften:

1. Das Markovische Prinzip besagt, dass die vergangenen States oder Aktionen keinen EInfluss auf die aktuelle oder zukünftigen Entscheidungen haben. Die Wahrscheinlichkeit, die die Transition-Funktion berechnet ist unabhängig von den vorherigen States.
2. Die Berechnung, die die Transitionsfunktion durchführt, ändert sich nicht.

Alle Schritte, die bis jetzt beschrieben wurden, befassen sich mit dem Problem. Um eine Lösung zu finden, wird eine Policy-Funktion pi(s) : a aufgestellt, die für einen State s eine Aktion zurückgibt, die die Höchste Belohnung für den Agenten vorhersagt. Die $Policy-Funktion$ kennt Tripel der Form <s,a,r> wobei s der aktuelle State ist, a die Aktion die im State s ausgeführt wird und r die Belohnung, die der Agent für die Aktion a im State s bekommen würde. Die Policy-Funktion hat viele von diesen Tripeln vorliegen und gibt die Aktion mit der höchsten Belohnung zurück. Eine optimale Policy pi* gibt immer die Aktion an, die langfristig die höchste Belohnung ergibt.

Der Prozess hat dabei die Markov-Eigenschaft. Das bedeutet, dass der Folgezustand nur vom aktuellen Zustand abhänging ist und nicht auf den vorangegangenen basiert. Es gilt also:

$P(S_{t+1}=s_t'|s_t,a_t,s_{t-1},a_{t-1})  =  P(S_{t+1}=s_t'|s_t,a_t) \forall s \in S$

Am Beispiel unseres Lieferwagen, wird das MDP wie folgt definiert:

## Definitionen
Wir bedienen zwei Lager und zwei Supermärkte.

Lager:
0. A-Lager (A)
1. B-Lager (B)

Supermarkt:
2. C-Markt (C)
3. D-Markt (D)

Diese verteilen sich folgendermaßen in unserer Stadt:

<img src="img/city.png" alt="stadt" width="400"/>

Die Stadt ist eine 6x6-Quadratestadt und mit 36 Positionen, die die Koordinaten $(0,0)$ bis $(5,5)$ haben. Der LKW kann sich frei in der Stadt bewegen, aber nicht durch die Grünstreifen fahren.

Die Anzahl der Zustände ergibt sich folgendermaßen:
* 6 x 6 Positionen
* 4 Orte zu denen die Ware gebracht werden kann (A bis D bzw. 0 bis 3)
* 5 Orte, an denen sich die Ware befindet (A bis D bzw. 0 bis 3 und im LKW (Position Nr.4))

\\[ 6 \cdot 6 \cdot 4 \cdot 5 = 720 \texttt{ mögliche Zustände}\\]

Die Aktionen, die der LKW ausführen kann sind:
0. nach Norden fahren
1. nach Osten fahren
2. nach Süden fahren
3. nach Westen fahren
4. Ware einsammeln
5. Ware abladen

Dabei kann er folgende Belohnungen (und Abzüge) erhalten:
* Ware korrekt abliefern: +20
* Ware falsch einsammeln/abliefern: -10
* Pro Schritt: -1

## Definitionen implementieren

In [ ]:
import copy
import random
import time

Im Folgenden werden die zuvor getroffenen Spezifikationen in ein für den Rechner verständliches Format überführt. Es gibt jeweils sechs Spalten und Reihen (0 bis 5) und eine Menge von Aktionen (ebenfalls 0 bis 5). Für die Ausgabe wird noch eine Überführung in eine Beschreibung vorgenommen. Außerdem werden die Koordinaten der Lager und Supermärlte festgehalten. Die Koordianten haben dabei die Form `(Reihe, Spalte)`.

Die Grünstreifen werden in einer Menge gespeichert. Ein Grünstreifen liegt immer zwischen zwei Feldern. Diese werden in einem Tupel in der Reihenfolge `(Links, Rechts)` bzw. `(Oben, Unten)` angegeben. Daraus ergibt sich für jedes Stück eines Grünstreifens ein Tupel der Form `((Reihe Zelle links, Spalte Zelle links), (Reihe Zelle rechts, Spalte Zelle rechts))` bzw. mit "oben" und "unten", falls es sich um einen horizontalen Streifen handelt.

In [1]:
rows = [row for row in range(0, 6)]
cols = [col for col in range(0, 6)]
actions = {action for action in range(0, 6)}
actions_description = ["Drive north", "Drive east", "Drive south", "Drive west", "Pickup goods", "Dropoff goods"]
stations = [(0,0), (0,3), (5,0), (4,5)]
stations_descriptions = ["Warehouse A", "Warehouse B", "Supermarket C", "Supermarket D"]
position_goods_descriptions = stations_descriptions + ["In the truck"]
walls = {
    ((0,2), (0,3)), #vertikl
    ((1,2), (1,3)),
    ((3,1), (3,2)),
    ((4,0), (4,1)),
    ((5,0), (5,1)),
    ((5,2), (5,3)),
    ((1,0), (2,0)), # horizonal
    ((1,5), (2,5)),
    ((3,4), (4,4)),
    ((3,5), (4,5))
}

...

## Transition function, Probability und Reward function
Die Transition function gibt an, mit welcher Wahrscheinlichkeit man von einem Zustand in den nächsten gelangt. Es handelt sich also um eine Funktion der Art  $P_{ss'}(s'| s, a)$. Dabei is $P$ die Wahrscheinlichkeit, dass Aktion $a$ vom Zustand $s$ zu $s'$ führt). Das Ergebnis der Funktion kann mit Hilfe einer Matrix dargestellt werden:


$P = \begin{bmatrix}
P_{11} & ... & P_{1n}\\
... &  &\\
P_{n1} & ... & P_{nn}
\end{bmatrix}$ wobei $1$ bis $n$ alle möglichen Zustände bezeichnen

Die Summe der Werte aus einer Reihe der Matrix muss für alle Reihen 1 ergeben.

Für jeden Zustandswechsel kann eine Belohnung $r$ ("Reward") vergeben werden. Der Höhe der Belohung ist durch die Reward function festgelegt. In vielen MDPs wird die Belohnung mit einem Faktor $\gamma$, dem "Discount" multipliziert. Dabei ist $\gamma \in [0,1]$ und wird mit der Anzahl der bisher erfolgten Schritte $t$ potenziert. Der Discount-Faktor wird oftmals genutzt, um zu erreichen, das früher erhaltene Belohnungen stärker gewichtet werden als spätere. Der aktuelle Wert des Zustands ergibt sich aus der Summe der erhaltenen Belohungen mal des Discounts. Ziel ist es, den Wert zu maximieren.
Es gilt:

\\[\texttt{Wert} = \sum_{k = 0}^t \gamma^k\cdot r_{k+1}\\]

